<a href="https://colab.research.google.com/github/superp0tat0/PlayGround/blob/master/Estimating_Gradients_for_Discrete_Random_Variables_by_Sampling_without_Replacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim 

from copy import deepcopy

import abstract_experiments_lib as ab_lib

import os
import sys

import optimization_lib as optim_lib
import baselines_lib as bs_lib
from scipy.stats import dirichlet

os.makedirs('plots', exist_ok=True)

ModuleNotFoundError: ignored

In [ ]:
pwd

'/content'

# The reimplementation and demonstration of Published Paper:

**Estimating Gradients For Discrete Random Variables By Sampling Without Replacement**

## Outline

The Key idea of this paper is the Rao Blackwellized estimators are better than the current estimators used in Gradient Estimation problems. Authors derived a novel, unbiased gradient estimator for discrete random variables based on sampling without replacement. They relate their estimator to existing multi-sample estimators and motivate why we would expect reduced variance. Finally, they evaluate their estimator across several tasks and show that it performs well in all of them.

In this notebook, we demonstrated the key idea of this paper is intuitive and easy to follow using an example related to genotype estimation based on blood type. Traditionally, this example could be solved by the Markov Chain Monte Carlo approach. In this case we choose Gibbs Sampler as our estimator. By comparison, we also derived the Rao Blackwellized version of Gibbs Sampler and showed the variance is significantly smaller and their biases are close.

We also reimplemented the algorithms demonstrated in paper and applied them on two different distributions. Such re-implementations verified Rao Blackwellized Estimators are better than most of the benchmark estimators in gradient estimation problems.

The authors also argue this estimator could perform well on both high and low entropy circumstances. However, we think such argument is fragile.

# Demonstration of the Key idea

## Rao Blackwell Theorem
If we have a estimator $g(X)$ of parameter $\theta$, Given a sufficient statistics $T(X)$. We could derive the Rao Blackwellized version of this estimator $g^*(X) = \mathbf{E}(g(X) | T(X))$ is a better estimator than g(X) itself of $\theta$, at least not worse due the following properties:


1. $Var(g^*(X)) \leq Var(g(X))$
1. If $g(X)$ is unbiased, so as $g^*(X)$.

We will use a simple example to demonstrate this idea


## Gibbs Sampler VS. Rao Blackwellized Gibbs Sampler

Suppose we sample some patients randomly from the hospital. We then collect the blood type of those patients and get the following data.
The first table showed the number of patients with the corresponding blood types

|  A | B  | AB  | O  |
|---|---|---|---|
|  724 | 158  | 87  | 787  |

The Second table showed the relations between the unobserved Genotype, the observed Blood Type and their probabilities.

|  Genotype | Probability  | Observed B-Type  | Probability  |
|---|---|---|---|
| AA  | $p_A^2$  | A  | $2p_Ap_O + p_A^2$  |
|  AO | $2p_Ap_O$  | A  | $2p_Ap_O + p_A^2$  |
| BB  | $p_B^2$  | B  | $2p_Bp_O + p_B^2$  |
|  BO | $2p_Bp_O$  |  B | $2p_Bp_O + p_B^2$  |
| AB  | $2p_Ap_B$  | AB  | $2p_Ap_B$  |
| OO  |  $p_O^2$ |  O | $p_O^2$   |

Assume $Z_1 = G(AA)$, $Z_2 = G(BB)$, $y_1 = B(AB)$, $y_2 = B(A)$, $y_3 = B(B)$ and $y_4 = B(O)$
Then

$\begin{align*}
    &P(\mathbf{Y, Z} | \mathbb{\theta}) &\\ &= 
    (p_Ap_B)^{y_1}p_A^{2Z_1}(p_Ap_O)^{y_2 - Z_1}p_B^{2Z_2}(p_Bp_O)^{y_3-Z_2}p_O^{2y_4} &\\
    &= p_A^{m_1}p_B^{m_2}p_O^{m_3}
\end{align*}$

where $m_1 = y_1 + y_2 + Z_1$, $m_2 = y_1 + y_3 + Z_2$ and $m_3 = y_2+y_3-Z_1-Z_2+2y_4$.

We then could use Gibbs Sampler to estimate the latent variables. The derived marginal distribution are given below, since we have no prior information we will set $a_1 = a_2 = a_3 = 1$
$$\mathbf{Z_1}|Y, p_A, p_B, p_O \sim Binom(y_2, \frac{p_A^2}{p_A^2 + 2p_A p_O})$$
$$\mathbf{Z_2}|Y, p_A, p_B, p_O \sim Binom(y_3, \frac{p_B^2}{p_B^2 + 2p_B p_O}) \\$$
$$\mathbf{\theta} = p_A,p_B,p_O \sim Dirichlet(m_1+a_1, m_2+a_2, m_3+a_3)$$

In [ ]:
def dirichlet(a, b, c):
    x1 = np.random.gamma(a, 1)
    x2 = np.random.gamma(b, 1)
    x3 = np.random.gamma(c, 1)
    s = x1 + x2 + x3
    return np.array([x1,x2,x3])/s

def RaoBlackwell(X1, X2, Z):
    result = (X1 + X2 + Z + 1) / (2*y1 + 2*y2 + 2*y3 + 2*y4 + 3)
    return result

In [ ]:
#Sample Amount
Sam = 3000

Z1 = np.array([0.0]*Sam)
Z2 = np.array([0.0]*Sam)
theta = np.array([1.0]*Sam*3).reshape((Sam,3))
y1 = 89; y2 = 642; y3 = 195; y4 = 657
Z1[0] = 500; Z2[0] = 500

theta[0] = [1/3, 1/3, 1/3]
a1 = 1; a2 = 1; a3 = 1

In [ ]:
for i in range(1,Sam):
    Z1[i] = np.random.binomial(y2,
                               (theta[i-1, 0]**2)/((theta[i-1, 0]**2) + 2*theta[i-1, 0]*theta[i-1, 2]), 1)
    Z2[i] = np.random.binomial(y3,
                               (theta[i-1, 1]**2)/((theta[i-1, 1]**2) + 2*theta[i-1, 1]*theta[i-1, 2]), 1)

    m1 = y1 + y2 + Z1[i]
    m2 = y1 + y3 + Z2[i]
    m3 = y2 + y3 - Z1[i] - Z2[i] + 2*y4

    theta[i, ] = dirichlet(m1+a1, m2+a2, m3+a3)

#Burn out the first 1000 samples
theta = theta[1000:,]

## Rao Blackwellized Gibbs Sampler

We then want to derive the Rao Blackwellized Gibbs Sampler using the following Bayesian formula. We assumed the sufficient statistics to be $\mathbf{Y}, \mathbf{Z}$

Then
$$P(p_A|p_B, p_O, \mathbf{Y}, \mathbf{Z}) = \frac{P(p_A, p_B, p_O | \mathbf{Y}, \mathbf{Z}) \times P(\mathbf{Y}, \mathbf{Z})}{P(p_B, p_O,\mathbf{Y}, \mathbf{Z})} =\frac{P(p_A, p_B, p_O|\mathbf{Y}, \mathbf{Z})}{P(p_B, p_O|\mathbf{Y}, \mathbf{Z})}$$


$$p_A|p_B, p_O, \mathbf{Y}, \mathbf{Z} \sim Beta(m_1+a_1, m_2+a_2 + m_3+a_3)$$

We could follow the same idea to derive the Monte Carlo estimation of $p_A$, $p_B$, and $p_O$.
$$\hat{p_A} = \frac{m_1 + 1}{m_1 + m_2 + m_3 + 3} = \frac{1}{M} \sum_i^M \frac{y_1 + y_2 + Z_{1i}+1}{2(y_1 + y_2 + y_3 + y_4)+3}$$
$$\hat{p_B} = \frac{m_2 + 1}{m_1 + m_2 + m_3 + 3} = \frac{1}{M} \sum_i^M \frac{y_1 + y_3 + Z_{2i}+1}{2(y_1 + y_2 + y_3 + y_4)+3}$$
$$\hat{p_O} = \frac{m_3 + 1}{m_1 + m_2 + m_3 + 3} = \frac{1}{M} \sum_i^M \frac{y_2 + y_3 - Z_{1i} - Z_{2i}+2y_4}{2(y_1 + y_2 + y_3 + y_4)+3}$$

The following plots showed the Rao BlackWellized Estimators have lower variance than the Gibbs Sampler Estimation

In [ ]:
fib, ax = plt.subplots()
sns.set_style('whitegrid')
sns.kdeplot(theta[:,0], bw=0.001, ax = ax, label="Gibbs")
sns.kdeplot(RaoBlackwell(y1, y2, Z1), bw=0.001, ax = ax, label="Rao Blackwellized Gibbs")
ax.set_title('Estiamtion of P(A)')
ax.set(xlabel="Probability", ylabel = "Frequency")
ax.legend()

In [ ]:
fib, ax = plt.subplots()
sns.set_style('whitegrid')
sns.kdeplot(theta[:,1], bw=0.001, ax = ax, label="Gibbs")
sns.kdeplot(RaoBlackwell(y1, y3, Z2), bw=0.001, ax = ax, label="Rao Blackwellized Gibbs")
ax.set_title('Estiamtion of P(B)')
ax.set(xlabel="Probability", ylabel = "Frequency")
ax.legend()

In [ ]:
fib, ax = plt.subplots()
sns.set_style('whitegrid')
sns.kdeplot(theta[:,2], bw=0.001, ax = ax, label="Gibbs")
sns.kdeplot(RaoBlackwell(y2+y3, 2*y4, -Z2-Z1), bw=0.001, ax = ax, label="Rao Blackwellized Gibbs")
ax.set_title('Estiamtion of P(O)')
ax.set(xlabel="Probability", ylabel = "Frequency")
ax.legend()

# Re-implementation

## The Unordered Set Policy Gradient Estimator

Our goal is to estimate the expectation of $f(\mathbf{x})$ where $\mathbf{x}$ has a discrete distribution $p$ over the domain $D$, i.e.

$$\mathbb{E}_{\mathbf{x} \sim p(\mathbf{x})}[f(\mathbf{x})] = \sum_{\mathbf{x} \in D}p(\mathbf{x})f(\mathbf{x})$$


Then let $S^k\subseteq D$ be an unordered sample without replacement from the distribtuion $p$, $s\in S^k$ be the elements in the sample, $p_{\boldsymbol{\theta}}$ indicates the dependency on the model parameters $\boldsymbol{\theta}$, define the $\textit{unordered set policy gradient estimator}$ as 

$$e^{U S P G}\left(S^{k}\right)=\sum_{s \in S^{k}} p_{\boldsymbol{\theta}}(s) R\left(S^{k}, s\right) \nabla_{\boldsymbol{\theta}} \log p_{\boldsymbol{\theta}}(s) f(s)=\sum_{s \in S^{k}} \nabla_{\boldsymbol{\theta}} p_{\boldsymbol{\theta}}(s) R\left(S^{k}, s\right) f(s)$$

where $\displaystyle R\left(S^{k}, s\right)=\frac{p^{D \backslash\{s\}}\left(S^{k} \backslash\{s\}\right)}{p\left(S^{k}\right)}$ is the leave-one-out ratio.

To reduce the variance, we can construct a baseline and substract it from $f$. The $\textit{unordered set policy gradient estimator with baseline}$ is given by

$$e^{U S P G B L}\left(S^{k}\right)=\sum_{s \in S^{k}} \nabla_{\boldsymbol{\theta}} p_{\boldsymbol{\theta}}(s) R\left(S^{k}, s\right)\left(f(s)-\sum_{s^{\prime} \in S^{k}} p_{\boldsymbol{\theta}}\left(s^{\prime}\right) R^{D \backslash\{s\}}\left(S^{k}, s^{\prime}\right) f\left(s^{\prime}\right)\right)$$

where $\displaystyle R^{D \backslash\{s\}}\left(S^{k}, s^{\prime}\right)=\frac{p_{\boldsymbol{\theta}}^{D \backslash\left\{s, s^{\prime}\right\}}\left(S^{k} \backslash\left\{s, s^{\prime}\right\}\right)}{p_{\boldsymbol{\theta}}^{D \backslash\{s\}}\left(S^{k} \backslash\{s\}\right)}$ is the second order leave-one-out ratio.

### Experiment

The experiment modifies the Bernoulli toy experiment in the paper "Estimating Gradients for Discrete Random Variables by Sampling without Replacement" and the code is adapted from https://github.com/wouterkool/estimating-gradients-without-replacement/tree/master/bernoulli from the GitHub of the paper. In this experiment, given a vector $\mathbf{p}=(0.6,0.51,0.48)$, the goal is to minimize

$$\mathbb{E}_{x_1,x_2,x_3\sim p(\sigma(\eta))}[\sum_{i=1}^3(x_i-p_i)^2]$$
where $x_1,x_2,x_3$ are i.i.d. from Geometric($\sigma(\eta)$)/Binominal(4,$\sigma(\eta)$) distribution and $\eta\in\mathbb{R}$, $\sigma(\eta)=(1+\exp(-\eta))^{-1}$ is the sigmoid function. We compare the variance of the unordered set policy gradient estimator with other estimators, with and without baseline, and set $\eta=0.0,4.0$.

We compare it with stratified/systematic sampling, VIMCO and other estimators, and present the plots of their gradient variance(log scale) for two distributions of different $\eta$ below. 

In [ ]:
softmax = nn.Softmax(dim = 0)
sigmoid = nn.Sigmoid()

In [ ]:
np.random.seed(454)
_ = torch.manual_seed(454)

In [ ]:
# fixed parameters
d = 3
# p0 = torch.rand(d)
p0 = torch.Tensor([0.6, 0.51, 0.48])
print('p0: ', p0, '\n')

print('sum(p0^2): ', torch.sum(p0**2))
print('sum((1 - p0)^2): ', torch.sum((1 - p0)**2), '\n')

# the optima
x_optimal = torch.argmin(torch.Tensor([torch.sum(p0**2), torch.sum((1 - p0)**2)]))

optimal_loss = torch.min(torch.Tensor([torch.sum(p0**2), torch.sum((1 - p0)**2)]))

print('optimal loss: ', optimal_loss)
print('optimal x: ', x_optimal.numpy())

In [ ]:
# random init for phi
phi0 = torch.Tensor([0.0])
phi0.requires_grad_(True)
print('init phi0: ', phi0)
print('init e_b: ', sigmoid(phi0))

In [ ]:
print(phi0)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
n_samples = 1000

def get_gradient_estimators(phi0,dist):
    phi0.requires_grad_(True)
    params = [phi0]
    experiment = ab_lib.BernoulliExperiments(p0, d, phi0)
    experiment.set_var_params(deepcopy(phi0))
    optbl = experiment.get_full_loss(dist).item()

    def get_grad_unordered(k, baseline_constant=None):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = 0, 
            grad_estimator = bs_lib.reinforce_unordered,
            n_samples = n_samples,
            grad_estimator_kwargs = {
                'n_samples': k,
                'baseline_constant': baseline_constant
            },dist=dist)

    def get_grad_unordered_optbl(k):
        return get_grad_unordered(k, baseline_constant=optbl)

    def get_grad_unordered_nobl(k):
        return get_grad_unordered(k, baseline_constant=0.0)
    
    def get_grad_stratified(k, systematic=False):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = 0, 
            grad_estimator = bs_lib.stratified,
            n_samples = n_samples,
            grad_estimator_kwargs = {
                'n_samples': k,
                'systematic': systematic
            },dist=dist)
    
    def get_grad_systematic(k):
        return get_grad_stratified(True)
    
    def get_grad_reinforce_wr(k, baseline_constant=None):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = 0, 
            grad_estimator = bs_lib.reinforce_wr,
            n_samples = n_samples,
            grad_estimator_kwargs = {
                'n_samples': k,
                'baseline_constant': baseline_constant
            },dist=dist)
    
    def get_grad_reinforce_wr_optbl(k):
        return get_grad_reinforce_wr(k, baseline_constant=optbl)
    
    def get_grad_reinforce_wr_nobl(k):
        return get_grad_reinforce_wr(k, baseline_constant=0.0)

    def get_grad_topk(k, sample_topk=False):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = k - 1, 
            sample_topk = sample_topk,
            grad_estimator = bs_lib.reinforce,
            n_samples = n_samples,
            grad_estimator_kwargs={'grad_estimator_kwargs': None},dist=dist)

    def get_grad_topk_bl(k, sample_topk=False):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = k - 1, 
            sample_topk = sample_topk,
            grad_estimator = bs_lib.reinforce_w_double_sample_baseline,
            n_samples = n_samples,
            grad_estimator_kwargs={'grad_estimator_kwargs': None},dist=dist)

    def get_grad_topk_optbl(k, sample_topk=False):
        return ab_lib.sample_gradient(
            phi0, experiment, 
            topk = k - 1, 
            grad_estimator = bs_lib.reinforce,
            n_samples = n_samples,
            grad_estimator_kwargs={'grad_estimator_kwargs': None, 'baseline': optbl},
            dist=dist)

    def get_grad_topk_sample(k):
        return get_grad_topk(k, sample_topk=True)

    def get_grad_topk_bl_sample(k):
        return get_grad_topk_bl(k, sample_topk=True)

    def get_grad_topk_optbl_sample(k):
        return get_grad_topk_optbl(k, sample_topk=True)

    alpha = 0.2

    xrng = np.arange(2 ** d) + 1
    n_eval = xrng
    n_eval_double = 2 * xrng
    n_eval_optbl = xrng + 2 ** d # Need all evaluations for baseline
    
    c_vimco = colors[1]
    c_detsas = colors[2]
    c_stochsas = colors[3]
    c_unord = colors[0]
    c_stratified = colors[4]
    c_systematic = colors[5]
    
    estimators = [
        
        (get_grad_stratified, "Stratified (no bl)", n_eval, {
            'color': c_stratified, 'linestyle': '--', 'marker': '.'}),
        (get_grad_systematic, "Systematic (no bl)", n_eval, {
            'color': c_systematic, 'linestyle': '--', 'marker': '.'}),
        (get_grad_reinforce_wr_nobl, "VIMCO (no bl)", n_eval, {
            'color': c_vimco, 'linestyle': '--', 'marker': '.'}),
        (get_grad_topk, "Det. sum & sample (no bl)", n_eval, {
            'color': c_detsas, 'linestyle': '--', 'marker': '.'}),
        (get_grad_topk_sample, "Stoch. sum & sample (no bl)", n_eval, {
            'color': c_stochsas, 'linestyle': '--', 'marker': '.'}),
        (get_grad_unordered_nobl, "Unordered (no bl)", n_eval, {
            'color': c_unord, 'linestyle': '--', 'marker': '.'}),
        
        (get_grad_reinforce_wr, "VIMCO (built-in bl)", n_eval, {
            'color': c_vimco, 'linestyle': '-', 'marker': '.'}),
        (get_grad_topk_bl, "Det. sum & sample (sample bl)", n_eval_double, {
            'color': c_detsas, 'linestyle': '-', 'marker': '.'}),
        (get_grad_topk_bl_sample, "Stoch. sum & sample (sample bl)", n_eval_double, {
            'color': c_stochsas, 'linestyle': '-', 'marker': '.'}),
        (get_grad_unordered, "Unordered (built-in bl)", n_eval, {
            'color': c_unord, 'linestyle': '-', 'marker': '.'}),
    ]
    return estimators

In [ ]:
def run_experiments(estimators, n_samples):
    mixed_grad_array_all = torch.zeros(len(estimators), (2**d), n_samples)

    for i in range(2**d):
        k = i + 1
        print(f"k = {k}")

        for j, (estimator, name, n_ev, plot_kwargs) in enumerate(estimators):
            print(name)
            try:
                grads = estimator(k)
                mixed_grad_array_all[j, i] = grads
            except:
                raise
                print(f"Warn: {name} with {k} evaluations failed")
    return mixed_grad_array_all

In [ ]:
def plot_results(estimators, mixed_grad_array_all, phi0, ylim=None, inc_baseline=False):

    mixed_grad_means = mixed_grad_array_all.mean(-1).numpy()
    mixed_grad_stds = mixed_grad_array_all.std(-1).numpy()
    
    mixed_grad_stds[mixed_grad_stds < 1e-8]  = 0

    fig = plt.figure(figsize=(8, 4))
    xrng = np.arange(mixed_grad_stds.shape[1]) + 1
    estimator_names = []
    for grad_arr, (est, name, n_ev, plot_kwargs) in zip(mixed_grad_stds, estimators):
        plt.plot(n_ev if inc_baseline else xrng, grad_arr ** 2, **plot_kwargs, lw='2')
        estimator_names.append(name)

    incl_text = "incl" if inc_baseline else "excl"
    plt.xlabel(f'Number of evaluations ({incl_text}. baseline)')
    plt.ylabel('Gradient variance (log scale)')
    if ylim is not None:
        plt.ylim(ylim)
    eta = phi0.detach().numpy()[0]
    plt.title('$\eta$ = {}'.format(eta))
    if ylim is None:
        plt.legend(estimator_names)
    plt.yscale('log')
    plt.tight_layout()
    plt.show()
    
    #fig.savefig("plots/toy_eta{}_{}.pdf".format(int(eta), "inc" if inc_baseline else "exc"),
    #            format='pdf')

In [ ]:
# run for eta=0.0 and 4.0 for Binominal Distribution
phi0 = torch.Tensor([0.0])
phi0_hard = torch.Tensor([-4.0])
estimators = get_gradient_estimators(phi0,dist='bin')
estimators_hard = get_gradient_estimators(phi0_hard,dist='bin')
mixed_grad_array_all = run_experiments(estimators, n_samples)
mixed_grad_array_all_hard = run_experiments(estimators_hard, n_samples)

In [ ]:
# plot the figure
inc_baseline=True
plot_results(estimators, mixed_grad_array_all, phi0, ylim=None, 
             inc_baseline=inc_baseline)
plot_results(estimators_hard, mixed_grad_array_all_hard, phi0_hard, ylim=None, 
             inc_baseline=inc_baseline)

In [ ]:
# run for eta=0.0 and 4.0 for Geometric Distribution
phi0 = torch.Tensor([0.0])
phi0_hard = torch.Tensor([-4.0])
estimators = get_gradient_estimators(phi0,dist='geo')
estimators_hard = get_gradient_estimators(phi0_hard,dist='geo')
mixed_grad_array_all = run_experiments(estimators, n_samples)
mixed_grad_array_all_hard = run_experiments(estimators_hard, n_samples)

In [ ]:
inc_baseline=True
plot_results(estimators, mixed_grad_array_all, phi0, ylim=None, 
             inc_baseline=inc_baseline)
plot_results(estimators_hard, mixed_grad_array_all_hard, phi0_hard, ylim=None, 
             inc_baseline=inc_baseline)

We can see from the above two graphs, as the number of evaluations increases, our estimator with baseline does have the lowest gradient variance among all the estimators, except for Binominal distribution at $\eta=4$ as the evaluation number is less than $8$. Use the experiments, we prove that the power of reducing the variance, however sometimes it depends on the model.

# Reference
1. Gibbs Sampler and Rao Blackwellization, math et al 2020-06-07: https://rpubs.com/mathetal/raoblackwellization
1. The Calculation of Posterior Distributions by Data Augmentation, Matin A. Tanner and Wing. H. Wong 2009-09-03
1. Bernoulli gradient variance, Wouter Kool, Herke van Hoof, Max Welling https://github.com/wouterkool/estimating-gradients-without-replacement/tree/master/bernoulli